Scattertext is a Python package that lets you interactively visualize how two categories of text are different from each other

### PytextRank + Scattertext

In [1]:
%pip install pytextrank -U
%pip install scattertext -U

  Using cached scipy-1.9.1-cp38-cp38-win_amd64.whl (38.6 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
causalnlp 0.6.0 requires Cython>=0.28.0, which is not installed.
gensim 4.2.0 requires Cython==0.29.28, which is not installed.
bioscrape 1.0.4 requires cython, which is not installed.
tigramite 5.0.1.16 requires numpy==1.21.5, but you have numpy 1.22.4 which is incompatible.
tigramite 5.0.1.16 requires scipy==1.8.0, but you have scipy 1.9.1 which is incompatible.
causalnlp 0.6.0 requires scipy==1.4.1, but you have scipy 1.9.1 which is incompatible.
causalnex 0.11.0 requires scikit-learn!=0.22.2.post1,!=0.24.1,<0.25.0,>=0.22.0; python_version < "3.9", but you have scikit-learn 1.1.2 which is incompatible.
causalnex 0.11.0 requires scipy<1.7,>=1.2.0, but you have scipy 1.9.1 which is incompatible.
vivarium-cobra 0.0.18 requires cobra==0.21.0, but you have cobra 0.5.4 which is incompatible.
scispacy 0.5.0 requires s

  Using cached scattertext-0.1.6-py3-none-any.whl (7.3 MB)
  Using cached Cython-0.29.28-py2.py3-none-any.whl (983 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
causalnlp 0.6.0 requires scipy==1.4.1, but you have scipy 1.9.1 which is incompatible.


https://medium.com/analytics-vidhya/visualizing-phrase-prominence-and-category-association-with-scattertext-and-pytextrank-f7a5f036d4d2#:~:text=Scattertext%20is%20a%20Python%20package,each%20other%20(Kessler%202017).

In [2]:
# !python -m spacy download en_core_web_sm

In [3]:
import pytextrank, spacy
import scattertext as st
import numpy as np
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("textrank", last=True)

In [4]:
convention_df = st.SampleCorpora.ConventionData2012.get_data(
).assign(
    parse=lambda df: df.text.apply(nlp),
    party=lambda df: df.party.apply(
        {'democrat': 'Democratic', 
         'republican': 'Republican'}.get
    )
)


In [5]:
convention_df.head()

,party,text,speaker,parse
0,Democratic,Thank you. Thank you. Thank you. Thank you so ...,BARACK OBAMA,"(Thank, you, ., Thank, you, ., Thank, you, ., ..."
1,Democratic,"Thank you so much. Tonight, I am so thrilled a...",MICHELLE OBAMA,"(Thank, you, so, much, ., Tonight, ,, I, am, s..."
2,Democratic,Thank you. It is a singular honor to be here t...,RICHARD DURBIN,"(Thank, you, ., It, is, a, singular, honor, to..."
3,Democratic,"Hey, Delaware. \nAnd my favorite Democrat, Jil...",JOSEPH BIDEN,"(Hey, ,, Delaware, ., \n, And, my, favorite, D..."
4,Democratic,"Hello. \nThank you, Angie. I'm so proud of how...",JILL BIDEN,"(Hello, ., \n, Thank, you, ,, Angie, ., I, 'm,..."


In [6]:
corpus = st.CorpusFromParsedDocuments(
    convention_df,
    category_col='party',
    parsed_col='parse',
    feats_from_spacy_doc=st.PyTextRankPhrases()
).build(
).compact(
    st.AssociationCompactor(2000, use_non_text_features=True)
)

c:\users\siddh\appdata\local\programs\python\python38\lib\site-packages\scattertext\termscoring\ScaledFScore.py:129: RuntimeWarning: invalid value encountered in true_divide
  precision = (cat_word_counts * 1. / (cat_word_counts + not_cat_word_counts))


In [7]:
term_category_scores = corpus.get_metadata_freq_df('')
print(term_category_scores.head(5))

                      Democratic  Republican
term                                        
new plants              0.104446    0.000000
jobs                    3.967825    2.860420
construction workers    0.120949    0.000000
more opportunity        0.175094    0.147941
more Americans          0.177554    0.054976


In [8]:
term_ranks = np.argsort(
    np.argsort(-term_category_scores, axis=0), 
axis=0) + 1
metadata_descriptions = {
    term: '<br/>' + '<br/>'.join(
        '<b>%s</b> TextRank score rank: %s/%s' % (
            cat, 
            term_ranks.loc[term, cat], 
            corpus.get_num_metadata()
        )
        for cat in corpus.get_categories()
    )
    for term in corpus.get_metadata()
}

In [9]:
category_specific_prominence = term_category_scores.apply(
    lambda row: (row.Democratic 
                 if row.Democratic > row.Republican 
                 else -row.Republican),
    axis=1
)

In [21]:
html = st.produce_scattertext_explorer(
    corpus,
    category='Democratic',
    minimum_term_frequency=0,
    pmi_threshold_coefficient=0,
    width_in_pixels=1000,
    transform=st.dense_rank,
    metadata=corpus.get_df()['speaker'],
    scores=category_specific_prominence,
    sort_by_dist=False,
    use_non_text_features=True,
    topic_model_term_lists={term: [term] for term in         
                            corpus.get_metadata()},
    topic_model_preview_size=0,
    metadata_descriptions=metadata_descriptions,
    use_full_doc=True
)

In [22]:
open("Convention-Visualization.html", 'wb').write(html.encode('utf-8'))

2361554

In [23]:
html = st.produce_scattertext_explorer(
    corpus,
    category='Democratic',
    minimum_term_frequency=0,
    pmi_threshold_coefficient=0,
    width_in_pixels=1000,
    transform=st.dense_rank,
    use_non_text_features=True,
    metadata=corpus.get_df()['speaker'],
    term_scorer=st.RankDifference(),
    sort_by_dist=False,
    topic_model_term_lists={term: [term] for term in 
                            corpus.get_metadata()},
    topic_model_preview_size=0, 
    metadata_descriptions=metadata_descriptions,
    use_full_doc=True
)

In [24]:
open("Convention-Visualization.html", 'wb').write(html.encode('utf-8'))

2361081

### PhraseMachine and Scattertext

In [26]:
import spacy
from scattertext import SampleCorpora, PhraseMachinePhrases, dense_rank, RankDifference, AssociationCompactor, produce_scattertext_explorer
from scattertext.CorpusFromPandas import CorpusFromPandas
corpus = CorpusFromPandas(
    SampleCorpora.ConventionData2012.get_data(),
    category_col='party',
    text_col='text',
    feats_from_spacy_doc=PhraseMachinePhrases(),
    nlp=spacy.load('en_core_web_sm')
).build().compact(AssociationCompactor(4000))
html = produce_scattertext_explorer(
    corpus,
    category='democrat',
    category_name='Democratic',
    not_category_name='Republican',
    minimum_term_frequency=0, 
    pmi_threshold_coefficient=0,
    transform=st.dense_rank,
    metadata=corpus.get_df()['speaker'],
    term_scorer=st.RankDifference(),
    width_in_pixels=1000
)

c:\users\siddh\appdata\local\programs\python\python38\lib\site-packages\scattertext\termscoring\ScaledFScore.py:296: RuntimeWarning: invalid value encountered in true_divide
  recall = cat_word_counts * 1. / cat_word_counts.sum()


In [27]:
open("Convention-Visualization.html", 'wb').write(html.encode('utf-8'))

1432355

### Visualize text embeddings

In [28]:
convention_df = st.SampleCorpora.ConventionData2012.get_data()
convention_df['parse'] = convention_df['text'].apply(st.whitespace_nlp_with_sentences)
corpus = (st.CorpusFromParsedDocuments(convention_df,
                                       category_col='party',
                                       parsed_col='parse')
                                       .build()
                                       .get_stoplisted_unigram_corpus()
                                       .remove_infrequent_words(minimum_term_count=3, 
                                                                term_ranker=st.OncePerDocFrequencyRanker))

In [29]:
from sklearn.feature_extraction.text import TfidfTransformer
embeddings = TfidfTransformer().fit_transform(corpus.get_term_doc_mat())
print(embeddings.shape)
print(corpus.get_num_docs(), corpus.get_num_terms())
embeddings = embeddings.T
embeddings.shape

(189, 2149)
189 2149


(2149, 189)

In [30]:
from scipy.sparse.linalg import svds
U, S, VT = svds(embeddings, k = 3, maxiter=20000, which='LM')
print(U.shape, S.shape, VT.shape)

(2149, 3) (3,) (3, 189)


In [31]:
import pandas as pd
x_dim = 0; y_dim = 1;
projection = pd.DataFrame({'term':corpus.get_terms(),
                            'x':U.T[x_dim],
                            'y':U.T[y_dim]}).set_index('term')

In [32]:
html = st.produce_pca_explorer(corpus,
                               category='democrat',
                               category_name='Democratic',
                               not_category_name='Republican',
                               projection=projection,
                               metadata=convention_df['speaker'],
                               width_in_pixels=1000,
                               x_dim=x_dim,
                               y_dim=y_dim)

In [33]:
open("Convention-Visualization.html", 'wb').write(html.encode('utf-8'))

1804886

In [34]:
html = st.produce_pca_explorer(corpus,
                               category='democrat',
                               category_name='Democratic',
                               not_category_name='Republican',
                               projection=projection,
                               metadata=convention_df['speaker'],
                               width_in_pixels=1000,
                               scaler=st.scale_neg_1_to_1_with_zero_mean,
                               x_dim=x_dim,
                               y_dim=y_dim)

In [35]:
open("Convention-Visualization.html", 'wb').write(html.encode('utf-8'))

1803849

### Topic modelling

In [36]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

convention_df = st.SampleCorpora.ConventionData2012.get_data()
convention_df['parse'] = convention_df['text'].apply(st.whitespace_nlp_with_sentences)

unigram_corpus = (st.CorpusFromParsedDocuments(convention_df,
                                               category_col='party',
                                               parsed_col='parse')
                  .build().get_stoplisted_unigram_corpus())
topic_model = st.SentencesForTopicModeling(unigram_corpus).get_topics_from_model(
	Pipeline([
		('tfidf', TfidfTransformer(sublinear_tf=True)),
		('nmf', (NMF(n_components=100, alpha=.1, l1_ratio=.5, random_state=0)))
	]),
	num_terms_per_topic=20
)
  
topic_feature_builder = st.FeatsFromTopicModel(topic_model)


topic_corpus = st.CorpusFromParsedDocuments(
	convention_df,
	category_col='party',
	parsed_col='parse',
	feats_from_spacy_doc=topic_feature_builder
).build()

html = st.produce_scattertext_explorer(
	topic_corpus,
	category='democrat',
	category_name='Democratic',
	not_category_name='Republican',
	width_in_pixels=1000,
	metadata=convention_df['speaker'],
	use_non_text_features=True,
	use_full_doc=True,
	pmi_threshold_coefficient=0,
	topic_model_term_lists=topic_feature_builder.get_top_model_term_lists(),
	topic_model_preview_size=20
)

C:\Users\siddh\AppData\Roaming\Python\Python38\site-packages\sklearn\decomposition\_nmf.py:1477: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(


In [37]:
open("Convention-Visualization.html", 'wb').write(html.encode('utf-8'))

1443560

In [39]:
convention_df = st.SampleCorpora.ConventionData2012.get_data()
convention_df['parse'] = convention_df['text'].apply(st.whitespace_nlp_with_sentences)

corpus = (st.CorpusFromParsedDocuments(convention_df, category_col='party', parsed_col='parse')
          .build().get_stoplisted_unigram_corpus())


In [42]:
# import umap
# from gensim.models.word2vec import Word2Vec

# html = st.produce_projection_explorer(corpus,
#                                       word2vec_model=Word2Vec(size=100, window=5, min_count=10, workers=4),
#                                       projection_model=umap.UMAP(min_dist=0.5, metric='cosine'),
#                                       category='democrat',
#                                       category_name='Democratic',
#                                       not_category_name='Republican',
#                                       metadata=convention_df.speaker)        

In [43]:
# open("Convention-Visualization.html", 'wb').write(html.encode('utf-8'))

### Lexicalized semiotic squares

In [44]:
import scattertext as st
movie_df = st.SampleCorpora.RottenTomatoes.get_data()
movie_df.category = movie_df.category.apply\
	(lambda x: {'rotten': 'Negative', 'fresh': 'Positive', 'plot': 'Plot'}[x])
corpus = st.CorpusFromPandas(
	movie_df,
	category_col='category',
	text_col='text',
	nlp=st.whitespace_nlp_with_sentences
).build().get_unigram_corpus()

semiotic_square = st.SemioticSquare(
	corpus,
	category_a='Positive',
	category_b='Negative',
	neutral_categories=['Plot'],
	scorer=st.RankDifference(),
	labels={'not_a_and_not_b': 'Plot Descriptions', 'a_and_b': 'Reviews'}
)

html = st.produce_semiotic_square_explorer(semiotic_square,
                                           category_name='Positive',
                                           not_category_name='Negative',
                                           x_label='Fresh-Rotten',
                                           y_label='Plot-Review',
                                           neutral_category_name='Plot Description',
                                           metadata=movie_df['movie_name'])

In [45]:
open("Convention-Visualization.html", 'wb').write(html.encode('utf-8'))

1713825

### Visualizing scikit-learn text classification weights

In [7]:
%pip install sklearn-contrib-lightning

     -------------------------------------- 644.0/644.0 kB 8.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from lightning.classification import CDClassifier
import scattertext as st

newsgroups_train = fetch_20newsgroups(
	subset='train',
	remove=('headers', 'footers', 'quotes')
)

In [9]:
vectorizer = TfidfVectorizer()
tfidf_X = vectorizer.fit_transform(newsgroups_train.data)
count_vectorizer = CountVectorizer(vocabulary=vectorizer.vocabulary_)

In [10]:
corpus = st.CorpusFromScikit(
	X=count_vectorizer.fit_transform(newsgroups_train.data),
	y=newsgroups_train.target,
	feature_vocabulary=vectorizer.vocabulary_,
	category_names=newsgroups_train.target_names,
	raw_texts=newsgroups_train.data
).build()

In [11]:
clf = CDClassifier(penalty="l1/l2",
                   loss="squared_hinge",
                   multiclass=True,
                   max_iter=20,
                   alpha=1e-4,
                   C=1.0 / tfidf_X.shape[0],
                   tol=1e-3)
clf.fit(tfidf_X, newsgroups_train.target)
term_scores = clf.coef_[0]

In [12]:
html = st.produce_frequency_explorer(
	corpus,
	'alt.atheism',
	scores=term_scores,
	use_term_significance=False,
	terms_to_include=st.AutoTermSelector.get_selected_terms(corpus, term_scores, 4000),
	metadata = ['/'.join(fn.split('/')[-2:]) for fn in newsgroups_train.filenames]
)

In [15]:
# newsgroups_test = fetch_20newsgroups(subset='test',
#                                      remove=('headers', 'footers', 'quotes'))
# X_test = vectorizer.transform(newsgroups_test.data)
# pred = clf.predict(X_test)
# f1 = f1_score(pred, newsgroups_test.target, average='micro')
# print("Microaveraged F1 score", f1)

In [13]:
open("Convention-Visualization.html", 'wb').write(html.encode('utf-8'))

16676890

### SentencePiece Tokens

In [9]:
import tempfile
import re
import scattertext as st

convention_df = st.SampleCorpora.ConventionData2012.get_data()
convention_df['parse'] = convention_df.text.apply(st.whitespace_nlp_with_sentences)

In [10]:
convention_df.head()

,party,text,speaker,parse
0,democrat,Thank you. Thank you. Thank you. Thank you so ...,BARACK OBAMA,"(thank, you, ., thank, you, ., thank, you, ., ..."
1,democrat,"Thank you so much. Tonight, I am so thrilled a...",MICHELLE OBAMA,"(thank, you, so, much, .)"
2,democrat,Thank you. It is a singular honor to be here t...,RICHARD DURBIN,"(thank, you, ., it, is, a, singular, honor, to..."
3,democrat,"Hey, Delaware. \nAnd my favorite Democrat, Jil...",JOSEPH BIDEN,"(hey, ,, delaware, ., and, my, favorite, democ..."
4,democrat,"Hello. \nThank you, Angie. I'm so proud of how...",JILL BIDEN,"(hello, ., thank, you, ,, angie, ., i, ', m, s..."


In [13]:
import sentencepiece as spm
def train_sentence_piece_tokenizer(documents, vocab_size):
    '''
    :param documents: list-like, a list of str documents
    :vocab_size int: the size of the vocabulary to output
    :return sentencepiece.SentencePieceProcessor
    '''
    sp = None
    with tempfile.NamedTemporaryFile(delete=True) as tempf:
        with tempfile.NamedTemporaryFile(delete=True) as tempm:
            tempf.write(('\n'.join(documents)).encode())
            mod = spm.SentencePieceTrainer.Train(input = tempf.name, model_prefix = tempm.name, vocab_size = vocab_size)
            sp = spm.SentencePieceProcessor()
            sp.load(tempm.name + '.model')
    return sp

In [ ]:
sp = train_sentence_piece_tokenizer(convention_df.text.values, 2000)

In [ ]:
corpus = st.CorpusFromParsedDocuments(
    convention_df,
    parsed_col='parse',
    category_col='party',
    feats_from_spacy_doc=st.FeatsFromSentencePiece(sp)
).build()

html = st.produce_scattertext_explorer(
    corpus,
    category='democrat',
    category_name='Democratic',
    not_category_name='Republican',
    sort_by_dist=False,
    metadata=convention_df['party'] + ': ' + convention_df['speaker'],
    term_scorer=st.RankDifference(),
    transform=st.Scalers.dense_rank,
    use_non_text_features=True,
    use_full_doc=True,
)